In [1]:
!pip install rouge_score evaluate pandas numpy transformers datasets

  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.0 MB/s eta 0:00:00
  Using cached datasets-4.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.6 MB/s eta 0:00:00
  Using cached xxhash-3.6.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.19-py312-none-any.whl.metadata (7.5 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.6 MB/s eta 0:00:00
  Using cached pyarrow-23.0.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.1 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached mu

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import re
import rouge_score
from evaluate import load
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, DataCollatorForSeq2Seq

/Users/agakshita/AI/FinTutor/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
import warnings
warnings.filterwarnings('ignore')

## dataset preprocess

In [10]:
dataset = load_dataset("bilalRahib/fiqa-personal-finance-dataset")
train_dataset = dataset["train"]
df1 = train_dataset.to_pandas()

Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 100%|██████████| 1987/1987 [00:00<00:00, 23424.09 examples/s]


In [11]:
df1.head()

,input,output
0,Does U.S. tax code call for small business own...,I am going to keep things very simple and expl...
1,What are 'business fundamentals'?,From http://financial-dictionary.thefreedictio...
2,Business Investment Loss from prior year,You need to give specific dates! In the United...
3,Would the purchase of a car for a business thr...,You don't say what country you live in. If it'...
4,Why is “cheque cashing” a legitimate business?,"In my experience (in the US), the main draw of..."


In [12]:
df2 = pd.read_csv('datasets/input_ds.csv')
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [13]:
df2.head()

,input,output
0,"Thanks. I'm pleased with the recent gains, but...",That's a valid concern. While the market's up ...
1,Should I be worried about recent market swings...,Current rate at 1.39% supports continued growt...
2,How does my moderate risk profile fit into thi...,"Your portfolio balances growth and stability, ..."
3,Are we positioned to capitalize on AI and rene...,"Absolutely, we’ve increased allocations in AI-..."
4,What about potential corrections—should I be w...,"While corrections are normal, our strategy emp..."


In [14]:
df = pd.concat([df1, df2])
df.head()

,input,output
0,Does U.S. tax code call for small business own...,I am going to keep things very simple and expl...
1,What are 'business fundamentals'?,From http://financial-dictionary.thefreedictio...
2,Business Investment Loss from prior year,You need to give specific dates! In the United...
3,Would the purchase of a car for a business thr...,You don't say what country you live in. If it'...
4,Why is “cheque cashing” a legitimate business?,"In my experience (in the US), the main draw of..."


In [15]:
df.shape

(18085, 2)

In [16]:
df.info()

<class 'pandas.DataFrame'>
Index: 18085 entries, 0 to 16097
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   input   18085 non-null  str  
 1   output  18051 non-null  str  
dtypes: str(2)
memory usage: 20.9 MB


In [17]:
df.dropna(axis=0, inplace= True)

In [18]:
df.shape

(18051, 2)

# saving clean data

In [20]:
def clean(text):
    text = text.strip()
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^a-zA-Z0-9.,?;:!%$€₹\- ]", "", text)  # keep basic punctuation
    text = text.lower()
    return text

df["input"] = df["input"].apply(clean)
df["output"] = df["output"].apply(clean)

df.to_csv("datasets/final_combined.csv", index=False)

In [21]:
df.head()

,input,output
0,does u.s. tax code call for small business own...,i am going to keep things very simple and expl...
1,what are business fundamentals?,from http:financial-dictionary.thefreedictiona...
2,business investment loss from prior year,you need to give specific dates! in the united...
3,would the purchase of a car for a business thr...,you dont say what country you live in. if its ...
4,why is cheque cashing a legitimate business?,"in my experience in the us, the main draw of c..."
